# Detectron2 Training

## Install requirements

In [ ]:
import sys
import os

In [ ]:
%pip install -r requirements.txt

In [ ]:
!python -m pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git

In [ ]:
sys.path.insert(0, os.path.abspath('./detectron2'))

In [ ]:
import torch
import detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

## Prepare dataset

In [ ]:
from roboflow import Roboflow

Replace these lines with your own credentials. For example:

rf = Roboflow(api_key='YOUR_API_KEY')

project = rf.workspace('YOUR_WORKSPACE').project('YOUR_PROJECT')

version = project.version(YOUR_VERSION_NUMBER)

dataset = version.download('YOUR_DATASET_FORMAT')

In [ ]:
rf = Roboflow(api_key="00000000000000000000")
project = rf.workspace("team").project("project")
version = project.version(0)
dataset = version.download("format:coco")

In [ ]:
dataPath = dataset.location + '/data.yaml'

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", dataPath+"/train")
register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", dataPath+"/valid")

## Training

In [ ]:
!detectron2/tools/train_net.py \
  --config-file detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml \
  --num-gpus 1 SOLVER.IMS_PER_BATCH 2 SOLVER.BASE_LR 0.0025

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output